In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
home_values = pd.read_csv(
    "C:\data\zillow\Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv"
)
rent_values = pd.read_csv(
    "C:\data\zillow\Metro_ZORI_AllHomesPlusMultifamily_Smoothed.csv"
)

In [ ]:
rent_values.head()

In [ ]:
home_values.head()

In [ ]:
valid_cities = [
    "New York, NY",
    "Chicago, IL",
    "Los Angeles-Long Beach-Anaheim, CA",
    "Philadelphia, PA",
    "Houston, TX",
    "Phoenix, AZ",
]

In [ ]:
selection = home_values["RegionName"].isin(valid_cities)
home_value_trimmed = home_values.loc[selection]

In [ ]:
selection = rent_values["RegionName"].isin(valid_cities)
rent_value_trimmed = rent_values.loc[selection]

In [ ]:
rent_value_trimmed

In [ ]:
x = ["SizeRank", "RegionID", "RegionType", "StateName"]
home_value_final = home_value_trimmed.drop(x, axis=1)

In [ ]:
columns = ["SizeRank", "RegionID"]
rent_value_final = rent_value_trimmed.drop(columns, axis=1)

In [ ]:
rent_value_final

In [ ]:
home_value_final

In [ ]:
ZHVI = pd.melt(
    home_value_final, id_vars=["RegionName"], value_name="ZHVI", var_name="Date"
)

In [ ]:
ZRI = pd.melt(
    rent_value_final, id_vars=["RegionName"], value_name="ZRI", var_name="Date"
)

In [ ]:
ZRI.tail()

In [ ]:
ZHVI.head()

In [ ]:
zillow_data = pd.merge(ZRI, ZHVI, on=["Date", "RegionName"], how="outer")

In [ ]:
zillow_data = zillow_data.sort_values("Date", ascending=True)

zillow_data

In [ ]:
zillow_data["Date"] = pd.to_datetime(zillow_data["Date"])

In [ ]:
zillow_data["Date"].head()

In [ ]:
zillow_data["Month"] = zillow_data["Date"].dt.month

In [ ]:
zillow_data["Year"] = zillow_data["Date"].dt.year

In [ ]:
zillow_data.head()

In [ ]:
annual_ZHVI = zillow_data.groupby(["RegionName", "Year"])["ZHVI"].mean()
annual_ZRI = zillow_data.groupby(["RegionName", "Year"])["ZRI"].mean()

In [ ]:
print(type(annual_ZHVI))

In [ ]:
annual_ZHVI.head()

In [ ]:
annual_ZRI.head()

In [ ]:
annual_ZHVI = annual_ZHVI.reset_index()
annual_ZRI = annual_ZRI.reset_index()

In [ ]:
annual_ZHVI.head(n=50)

In [ ]:
with plt.style.context("ggplot"):
    fig, ax = plt.subplots(figsize=(10, 6))
    for city in annual_ZHVI["RegionName"].unique():
        selection = annual_ZHVI["RegionName"] == city
        df = annual_ZHVI.loc[selection]
        ax.plot(df["Year"], df["ZHVI"] / 1000, label=city, linewidth=5)
    ax.set_ylim(50, 850)
    ax.legend(loc=0, ncol=2, fontsize=10)
    ax.set_ylabel("Zillow Home Value Index\n(in thousands of dollars)")
    ax.set_xlabel("Year")

In [ ]:
with plt.style.context("ggplot"):
    fig, ax = plt.subplots(figsize=(10, 6))
    for city in annual_ZRI["RegionName"].unique():
        selection = annual_ZRI["RegionName"] == city
        df = annual_ZRI.loc[selection]
        ax.plot(df["Year"], df["ZRI"], label=city, linewidth=5)
    ax.set_ylim(1000, 3300)
    ax.legend(loc=0, ncol=2, fontsize=10)
    ax.set_ylabel("Zillow Rent Index(in dollars)")
    ax.set_xlabel("Year")